To run this Application write this in your terminal:

```py -3.10 -m venv myenv```

```.\myenv\Scripts\Activate```

select the kernal to be myenv (Python 3.10.0)

```py --version``` insure this version is 3.10.0


In [111]:
!py --version

Python 3.10.0


- **this cell could take some time to run**

In [131]:
import subprocess
import importlib
import os
import json
from pathlib import Path
import logging
from concurrent.futures import ThreadPoolExecutor 
    
# Define the libraries and their pip install commands
libraries = {
    "tensorflow": "pip install tensorflow",
    "tensorflow-hub": "pip install tensorflow-hub",
    "numpy": "pip install numpy",
    "matplotlib": "pip install matplotlib",
    "pandas": "pip install pandas",
    "cv2": "pip install opencv-python", 
    "keras": "pip install keras",
    "pillow": "pip install pillow",
}

for library, command in libraries.items():
    try:
        importlib.import_module(library)
        print(f"{library} is already installed.")
    except ImportError:
        print(f"{library} not found. Installing...")
        try:
            subprocess.run(command.split(), check=True)
            print(f"{library} installed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {library}. Error: {e}")


import tensorflow as tf
import tensorflow_hub as hub
from keras import models, layers, applications, utils, preprocessing
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

tensorflow is already installed.
tensorflow-hub not found. Installing...
tensorflow-hub installed successfully.
numpy is already installed.
matplotlib is already installed.
pandas is already installed.
cv2 is already installed.
keras is already installed.
pillow not found. Installing...
pillow installed successfully.


In [113]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

In [114]:
def video_to_frames(video_path, output_folder, frame_rate=1):
    """
    Extracts frames from a video at a specified frame rate.

    Args:
        video_path (str): Path to the video file.
        output_folder (str): Folder to save the extracted frames.
        frame_rate (int): Number of frames to save per second of video.

    Returns:
        None
    """
    if frame_rate <= 0:
        logging.error("Error: Frame rate must be a positive integer.")
        return

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        logging.error(f"Error: Could not open video file '{video_path}'.")
        return

    # Get the video's frame rate and total number of frames
    video_fps = int(video.get(cv2.CAP_PROP_FPS))
    if video_fps <= 0:
        logging.error("Error: Invalid video FPS. Please check the video file.")
        video.release()
        return

    if frame_rate > video_fps:
        logging.warning(f"Warning: Frame rate ({frame_rate}) exceeds video FPS ({video_fps}). Adjusting to match video FPS.")
        frame_rate = video_fps

    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    logging.info(f"Video FPS: {video_fps}, Total Frames: {frame_count}")

    # Calculate the frame interval for the desired frame rate
    frame_interval = video_fps // frame_rate

    # Extract and save frames
    frame_idx = 0
    saved_frame_idx = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break  # End of video

        # Save the frame at the specified interval
        if frame_idx % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_frame_idx:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_frame_idx += 1

        frame_idx += 1

    # Release the video capture object
    video.release()

    if saved_frame_idx == 0:
        logging.warning("No frames were saved. Adjust the frame rate and try again.")
    else:
        logging.info(f"{saved_frame_idx} frames saved to '{output_folder}'.")
        
    return saved_frame_idx

In [115]:
target_extraction_folders = [
    "workspace/good_pose",
    "workspace/bad_pose",
    "workspace/cant_determine"
]


def process_session(session_path, label, output_dir, frame_rate=1):
    """
    Processes a single session folder, extracting frames and mapping to sensor data.
    """
    video_path, json_path = None, None

    for file_name in os.listdir(session_path):
        file_path = os.path.join(session_path, file_name)
        lower = file_name.lower()
        if lower.endswith('.mp4'):
            video_path = file_path
        elif lower.endswith('.json'):
            json_path = file_path

    if not video_path or not os.path.exists(video_path):
        logging.warning(f"No valid video found in: {session_path}")
        return
    if not json_path or not os.path.exists(json_path):
        logging.warning(f"No valid JSON file found in: {session_path}")
        return

    with open(json_path, 'r') as f:
        sensor_data = json.load(f)

    gyro_list = sensor_data.get("gyroscopeData", [])
    accel_list = sensor_data.get("accelerometerData", [])

    session_name = Path(session_path).name
    session_output = os.path.join(output_dir, label, session_name)
    os.makedirs(session_output, exist_ok=True)

    saved_frames = video_to_frames(video_path, session_output, frame_rate)
    if saved_frames == 0:
        logging.warning("No frames were extracted; skipping sensor mapping.")
        return

    if not gyro_list or not accel_list:
        logging.warning("Missing sensor data in JSON; skipping sensor mapping.")
        return

    frame_files = sorted([f for f in os.listdir(session_output) if f.lower().endswith('.jpg')])
    csv_path = os.path.join(session_output, "frame_sensor_mapping.csv")

    with open(csv_path, 'w') as csv_file:
        csv_file.write("frame_file,frame_idx,sensor_idx,gyro_x,gyro_y,gyro_z,accel_x,accel_y,accel_z\n")
        M = saved_frames
        N = len(gyro_list)
        L = len(accel_list)

        for i, frame_file in enumerate(frame_files):
            gyro_idx = int((i / M) * N)
            accel_idx = int((i / M) * L)

            gyro_data = gyro_list[gyro_idx]
            accel_data = accel_list[accel_idx]

            csv_file.write(
                f"{frame_file},{i},{gyro_idx},"
                f"{gyro_data.get('x', 0)},{gyro_data.get('y', 0)},{gyro_data.get('z', 0)},"
                f"{accel_data.get('x', 0)},{accel_data.get('y', 0)},{accel_data.get('z', 0)}\n"
            )

            # old_path = os.path.join(session_output, frame_file)
            # base, ext = os.path.splitext(frame_file)
            # new_filename = f"{base}_sensor_{gyro_idx}{ext}"
            # new_path = os.path.join(session_output, new_filename)
            # os.rename(old_path, new_path)

    logging.info(f"Frame-to-sensor mapping complete for: {session_path}")

In [116]:
output_dir = "extracted_frames"
# frame_rate = 30
frame_rate = 1 # for demonstration purposes

def process_label(label):
        label_path = Path(label)
        if not label_path.exists():
            logging.warning(f"Label folder not found: {label_path}")
            return

        for session_name in os.listdir(label_path):
            session_path = os.path.join(label_path, session_name)
            if not os.path.isdir(session_path):
                continue

            try:
                process_session(session_path, label, output_dir, frame_rate)
            except Exception as e:
                logging.error(f"Error processing session '{session_path}': {e}")

with ThreadPoolExecutor() as executor:
    executor.map(process_label, target_extraction_folders)

2025-01-08 16:04:00,754 - INFO - Video FPS: 30, Total Frames: 525
2025-01-08 16:04:00,766 - INFO - Video FPS: 29, Total Frames: 1217
2025-01-08 16:04:00,790 - INFO - Video FPS: 29, Total Frames: 751
2025-01-08 16:04:04,841 - INFO - 18 frames saved to 'extracted_frames\workspace/bad_pose\10'.
2025-01-08 16:04:04,845 - WARNING - Missing sensor data in JSON; skipping sensor mapping.
2025-01-08 16:04:04,905 - INFO - Video FPS: 29, Total Frames: 311
2025-01-08 16:04:06,659 - INFO - 26 frames saved to 'extracted_frames\workspace/good_pose\1'.
2025-01-08 16:04:06,664 - INFO - Frame-to-sensor mapping complete for: workspace\good_pose\1
2025-01-08 16:04:06,716 - INFO - Video FPS: 29, Total Frames: 667
2025-01-08 16:04:07,311 - INFO - 11 frames saved to 'extracted_frames\workspace/bad_pose\12'.
2025-01-08 16:04:07,316 - INFO - Frame-to-sensor mapping complete for: workspace\bad_pose\12
2025-01-08 16:04:07,367 - INFO - Video FPS: 29, Total Frames: 1018
2025-01-08 16:04:10,273 - INFO - 42 frames s

In [117]:
import pandas as pd
mapped_df = pd.read_csv("extracted_frames/workspace/good_pose/1/frame_sensor_mapping.csv")
mapped_df.head()

,frame_file,frame_idx,sensor_idx,gyro_x,gyro_y,gyro_z,accel_x,accel_y,accel_z
0,frame_0000.jpg,0,0,-0.384450,0.453337,0.583412,-0.05100,7.07805,-7.16805
1,frame_0001.jpg,1,99,-0.007425,-0.029288,0.002887,-0.31200,7.41105,-6.30105
2,frame_0002.jpg,2,198,-0.036162,-0.098587,-0.021588,-0.23700,7.63905,-5.96805
3,frame_0003.jpg,3,297,0.049087,-0.113437,-0.003437,-0.54900,7.70595,-5.63295
4,frame_0004.jpg,4,396,-0.060638,0.000550,0.007150,-0.49095,7.80195,-5.57295


In [118]:
IMAGE_SIZE = (224, 224)
DATA_DIR = "extracted_frames/workspace"
DATA_COLS = ["gyro_x", "gyro_y", "gyro_z", "accel_x", "accel_y", "accel_z"]

In [ ]:
import tensorflow_addons as tfa
# ModuleNotFoundError: No module named 'keras.src.engine'Cell Execution Error

def augment_image(img):
    """
    Apply enhanced data augmentation to a single image.

    Args:
        img (np.ndarray): Input image as a NumPy array.

    Returns:
        np.ndarray: Augmented image as a NumPy array.
    """
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)

    # Basic augmentations
    img_tensor = tf.image.random_flip_left_right(img_tensor)  # Random horizontal flip
    img_tensor = tf.image.random_flip_up_down(img_tensor)  # Random vertical flip
    img_tensor = tf.image.random_brightness(img_tensor, max_delta=0.2)  # Random brightness
    img_tensor = tf.image.random_contrast(img_tensor, lower=0.8, upper=1.2)  # Random contrast

    # Rotation
    angle = tf.random.uniform([], minval=-15, maxval=15, dtype=tf.float32)  # Random rotation (-15° to +15°)
    img_tensor = tfa.image.rotate(img_tensor, angles=tf.constant(angle), interpolation='BILINEAR')

    # Shear Transformation
    img_tensor = tfa.image.shear_x(img_tensor, level=tf.random.uniform([], -0.3, 0.3), replace_value=0)
    img_tensor = tfa.image.shear_y(img_tensor, level=tf.random.uniform([], -0.3, 0.3), replace_value=0)

    # Scaling and Cropping
    img_tensor = tf.image.resize_with_crop_or_pad(img_tensor, IMAGE_SIZE[0] + 20, IMAGE_SIZE[1] + 20)  # Add padding
    img_tensor = tf.image.random_crop(img_tensor, size=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))  # Random crop

    # Color Adjustments
    img_tensor = tf.image.random_hue(img_tensor, max_delta=0.05)  # Adjust hue
    img_tensor = tf.image.random_saturation(img_tensor, lower=0.8, upper=1.2)  # Adjust saturation

    # Clip values to valid range
    img_tensor = tf.clip_by_value(img_tensor, 0.0, 1.0)  # Clip values between 0 and 1 (normalized range)

    return img_tensor.numpy()

ModuleNotFoundError: No module named 'keras.src.engine'

In [145]:
def load_data(data_dir, min_frames_per_session=1, augmentations_per_image=3):
    """
    Loads images, sensor data, and labels from the dataset directory, with data augmentation.

    Args:
        data_dir (str): Path to the dataset directory.
        min_frames_per_session (int): Minimum number of frames required to process a session.
        augmentations_per_image (int): Number of augmented images to generate per original image.

    Returns:
        Tuple: (images, sensors, labels)
    """
    images = []
    sensors = []
    labels = []

    for label_folder in ["good_pose", "bad_pose"]:
        label_path = os.path.join(data_dir, label_folder)
        if not os.path.exists(label_path):
            logging.warning(f"Label folder not found: {label_path}")
            continue

        # Loop through each session folder
        session_folders = [f for f in os.listdir(label_path) if os.path.isdir(os.path.join(label_path, f))]
        for session in session_folders:
            session_path = os.path.join(label_path, session)

            # Find the sensor mapping CSV
            csv_file = os.path.join(session_path, "frame_sensor_mapping.csv")
            if not os.path.exists(csv_file):
                logging.warning(f"Sensor mapping CSV not found: {csv_file}")
                continue

            # Load the sensor data from CSV
            df = pd.read_csv(csv_file)
            session_images = []
            session_sensors = []
            session_labels = []

            # Get all available frame filenames in the session directory
            available_frames = {os.path.basename(f) for f in os.listdir(session_path) if f.endswith('.jpg')}
            logging.debug(f"Available frames in {session_path}: {available_frames}")

            for _, row in df.iterrows():
                frame_file_name = row["frame_file"]
                if frame_file_name not in available_frames:
                    logging.warning(f"Frame file not found: {frame_file_name}")
                    continue

                frame_file_path = os.path.join(session_path, frame_file_name)
                try:
                    # Load and preprocess image
                    img = preprocessing.image.load_img(frame_file_path, target_size=IMAGE_SIZE)
                    img = preprocessing.image.img_to_array(img) / 255.0  # Normalize to [0, 1]

                    # Apply augmentations
                    augmented_imgs = [img] + [augment_image(img) for _ in range(augmentations_per_image)]

                    # Load sensor data
                    sensor_features = np.array([
                        row["gyro_x"], row["gyro_y"], row["gyro_z"],
                        row["accel_x"], row["accel_y"], row["accel_z"]
                    ])

                    # Append data
                    for augmented_img in augmented_imgs:
                        session_images.append(augmented_img)
                        session_sensors.append(sensor_features)
                        session_labels.append(1 if label_folder == "good_pose" else 0)
                except Exception as e:
                    logging.error(f"Error processing frame {frame_file_path}: {e}")
                    continue

            # Check if session has enough frames
            if len(session_images) < min_frames_per_session:
                logging.warning(f"Session skipped due to insufficient frames: {session_path}")
                continue

            # Add session data to global lists
            images.extend(session_images)
            sensors.extend(session_sensors)
            labels.extend(session_labels)

    images = np.array(images, dtype=np.float32)
    sensors = np.array(sensors, dtype=np.float32)
    labels = utils.to_categorical(np.array(labels, dtype=np.int32), num_classes=2)

    logging.info(f"Loaded {len(labels)} samples (including augmented) from {data_dir}.")
    return images, sensors, labels

In [153]:
img = preprocessing.image.load_img('extracted_frames/workspace/bad_pose/10/frame_0000.jpg', target_size=IMAGE_SIZE)
# print(img)
img_array = preprocessing.image.img_to_array(img)
# print(img_array)
augmented_imgs = [img_array] + [augment_image(img) for _ in range(3)]

# store teh augmented images in a directory
augmented_dir = "augmented_images"
os.makedirs(augmented_dir, exist_ok=True)
for i, img in enumerate(augmented_imgs):
    img_path = os.path.join(augmented_dir, f"augmented_{i}.jpg")
    img_pil = preprocessing.image.array_to_img(img)
    img_pil.save(img_path)

In [126]:
def split_data(images, sensors, labels, validation_split=0.2, test_split=0.1):
    """
    Splits the dataset into training, testing, and validation sets.
    
    Args:
        images (np.ndarray): Array of image data.
        sensors (np.ndarray): Array of sensor data.
        labels (np.ndarray): Array of labels.
        validation_split (float): Proportion of the data to use for validation.
        test_split (float): Proportion of the data to use for testing.
        
    Returns:
        Tuple: (X_img_train, X_img_val, X_img_test, 
                X_sensor_train, X_sensor_val, X_sensor_test, 
                y_train, y_val, y_test)
    """
    dataset_size = len(images)
    test_size = int(dataset_size * test_split)
    val_size = int(dataset_size * validation_split)
    train_size = dataset_size - test_size - val_size

    # Shuffle indices
    indices = np.arange(dataset_size)
    np.random.shuffle(indices)

    # Split indices
    test_indices = indices[:test_size]
    val_indices = indices[test_size:test_size + val_size]
    train_indices = indices[test_size + val_size:]

    # Slice data
    X_img_train = images[train_indices]
    X_img_val = images[val_indices]
    X_img_test = images[test_indices]
    X_sensor_train = sensors[train_indices]
    X_sensor_val = sensors[val_indices]
    X_sensor_test = sensors[test_indices]
    y_train = labels[train_indices]
    y_val = labels[val_indices]
    y_test = labels[test_indices]

    return (X_img_train, X_img_val, X_img_test, 
            X_sensor_train, X_sensor_val, X_sensor_test, 
            y_train, y_val, y_test)


In [146]:
images, sensors, labels = load_data(DATA_DIR)
# Split data using custom splitting logic

print(
    f"Loaded data: Images - {images.shape}, Sensors - {sensors.shape}, Labels - {labels.shape}"
)

2025-01-08 17:13:00,135 - WARNING - Sensor mapping CSV not found: extracted_frames/workspace\good_pose\11\frame_sensor_mapping.csv
2025-01-08 17:13:54,942 - WARNING - Sensor mapping CSV not found: extracted_frames/workspace\bad_pose\10\frame_sensor_mapping.csv
2025-01-08 17:14:32,042 - WARNING - Sensor mapping CSV not found: extracted_frames/workspace\bad_pose\9\frame_sensor_mapping.csv
2025-01-08 17:16:19,867 - INFO - Loaded 15916 samples (including augmented) from extracted_frames/workspace.


Loaded data: Images - (15916, 224, 224, 3), Sensors - (15916, 6), Labels - (15916, 2)


In [128]:
X_img_train, X_img_val, X_img_test, X_sensor_train, X_sensor_val, X_sensor_test, y_train, y_val, y_test = split_data(
    images, sensors, labels, validation_split=0.2, test_split=0.1
)

# Check shapes
print("Image training data shape:", X_img_train.shape)
print("Sensor training data shape:", X_sensor_train.shape)
print("Label training data shape:", y_train.shape)
print("Image validation data shape:", X_img_val.shape)
print("Sensor validation data shape:", X_sensor_val.shape)
print("Label validation data shape:", y_val.shape)
print("Image testing data shape:", X_img_test.shape)
print("Sensor testing data shape:", X_sensor_test.shape)
print("Label testing data shape:", y_test.shape)

Image training data shape: (2787, 224, 224, 3)
Sensor training data shape: (2787, 6)
Label training data shape: (2787, 2)
Image validation data shape: (795, 224, 224, 3)
Sensor validation data shape: (795, 6)
Label validation data shape: (795, 2)
Image testing data shape: (397, 224, 224, 3)
Sensor testing data shape: (397, 6)
Label testing data shape: (397, 2)


In [141]:
from PIL import Image
import matplotlib.pyplot as plt
X_img_train[0]

array([[[0.75686276, 0.7529412 , 0.7372549 ],
        [0.7529412 , 0.7490196 , 0.73333335],
        [0.7607843 , 0.74509805, 0.73333335],
        ...,
        [0.8117647 , 0.80784315, 0.7921569 ],
        [0.8156863 , 0.8117647 , 0.79607844],
        [0.81960785, 0.8156863 , 0.8       ]],

       [[0.76862746, 0.7529412 , 0.7411765 ],
        [0.76862746, 0.7529412 , 0.7411765 ],
        [0.76862746, 0.7529412 , 0.7411765 ],
        ...,
        [0.81960785, 0.8156863 , 0.8       ],
        [0.8235294 , 0.81960785, 0.8039216 ],
        [0.81960785, 0.8156863 , 0.8       ]],

       [[0.77254903, 0.7529412 , 0.7411765 ],
        [0.77254903, 0.7529412 , 0.7411765 ],
        [0.76862746, 0.75686276, 0.7372549 ],
        ...,
        [0.83137256, 0.8156863 , 0.8039216 ],
        [0.8235294 , 0.81960785, 0.8039216 ],
        [0.8235294 , 0.81960785, 0.8039216 ]],

       ...,

       [[0.16862746, 0.17254902, 0.1882353 ],
        [0.16078432, 0.16470589, 0.18039216],
        [0.16470589, 0

In [122]:
# Image input
image_input = layers.Input(shape=(224, 224, 3))
base_model = applications.MobileNetV2(weights='imagenet', include_top=False, input_tensor=image_input)
image_features = layers.GlobalAveragePooling2D()(base_model.output)

# Sensor input
sensor_input = layers.Input(shape=(6,))  # 3 gyroscope + 3 accelerometer features
sensor_features = layers.Dense(64, activation='relu')(sensor_input)

# Combine features
combined = layers.concatenate([image_features, sensor_features])
combined = layers.Dense(128, activation='relu')(combined)
output = layers.Dense(2, activation='softmax')(combined)

model = models.Model(inputs=[image_input, sensor_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\obada\AppData\Local\Temp\ipykernel_13032\2539727540.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = applications.MobileNetV2(weights='imagenet', include_top=False, input_tensor=image_input)
